In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [2]:
from warnings import filterwarnings
filterwarnings('ignore')

In [3]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional, Dropout, LSTM
from tensorflow.keras.layers import Embedding
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import sparse_categorical_crossentropy

In [4]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
#from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

from sklearn.pipeline import Pipeline, make_union, make_pipeline
from sklearn.cluster import KMeans


from nltk.translate.bleu_score import sentence_bleu
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer 
import nltk
from nltk import FreqDist

import tensorflow as tf
#from tensorflow.keras.model

### Loading The Data

In [5]:
Train = pd.read_csv('Train.csv')
Test = pd.read_csv('Test.csv')

In [6]:
def get_len(snt):
    return len(nltk.word_tokenize(snt))
    

#### Preprocessing

In [7]:
def Preprocess():
    #English
    global eng_vocabs, yor_vocabs
    eng_tokenizer = Tokenizer()
    eng_tokenizer.fit_on_texts(Train['English'])
    dist_eng = eng_tokenizer.word_counts
    eng_vocabs = eng_tokenizer.word_index.keys()

    #Yoruba
    yor_tokenizer = Tokenizer()
    yor_tokenizer.fit_on_texts(Train['Yoruba'])
    dist_yor = yor_tokenizer.word_counts
    yor_vocabs = yor_tokenizer.word_index.keys()
    
    #get lenght
    global yor_maxlen, eng_maxlen
    yor_maxlen = Train ['Yoruba'].apply(get_len).max()
    eng_maxlen = Train['English'].apply(get_len).max()
    
    #make_Xy(yor_vocabs,series_tokenize(sentences), wordy_idx, yor_len)
    global X_train, y_train, X_test, y_test
    x = eng_tokenizer.texts_to_sequences(Train['English'])
    X = pad_sequences(x, maxlen = eng_maxlen, padding='post')
    X = X.reshape((*X.shape,1))
    X_train = X[:9000]
    X_test = X[9000:]
    
    Y = yor_tokenizer.texts_to_sequences(Train['Yoruba'])
    y = pad_sequences(Y, maxlen = yor_maxlen, padding='post')
    y = y.reshape(*y.shape, 1)
    y_train = y[:9000]
    y_test = y[9000:]
    
   
    #x_test = eng_tokenizer.texts_to_sequences(Train['English'].loc[9001:])
    #Y_test = yor_tokenizer.texts_to_sequences(Train['Yoruba'].loc[9001:])
    
    
    


In [8]:
Preprocess()

In [104]:
X_train.shape

(9000, 200, 1)

In [105]:
y_train.shape

(9000, 243, 1)

In [107]:
input_shape = X_train.shape
# Hyperparameters
learning_rate = 0.005

# TODO: Build the layers
model = Sequential()
model.add(GRU(256, input_shape=input_shape[1:], return_sequences=True))
model.add(TimeDistributed(Dense(1024, activation='relu')))
model.add(Dropout(0.5))
model.add(TimeDistributed(Dense(yor_maxlen, activation='softmax'))) 

# Compile model
model.compile(loss=sparse_categorical_crossentropy,
              optimizer=Adam(learning_rate),
                  metrics=['accuracy'])

In [108]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_3 (GRU)                  (None, 200, 256)          198144    
_________________________________________________________________
time_distributed_2 (TimeDist (None, 200, 1024)         263168    
_________________________________________________________________
dropout_1 (Dropout)          (None, 200, 1024)         0         
_________________________________________________________________
time_distributed_3 (TimeDist (None, 200, 243)          249075    
Total params: 710,387
Trainable params: 710,387
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(X_train, y_train, batch_size=1024, epochs = 20, validation_split=0.2)

Train on 7200 samples, validate on 1800 samples
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/20
